<h1>Setting up everything

<h2>Imports


In [ ]:
# spotipy modules
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
# other libraries
from decouple import config 
import time
from IPython.core.display import clear_output
import csv
from itertools import chain
import string
from random import sample

In [ ]:
# visualisation libraries
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# increase number of visible rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 20)

pd.set_option('display.max_columns', None)

## Setting up oauth object

In [ ]:
# environment variables
import os

os.environ['CLIENT_ID'] = '0e1a2088928f483c8a86bfff72efb84e'
os.environ['CLIENT_SECRET'] = 'a37836eb01954efcab1a9c1dbf7beb84'
os.environ['REDIRECT_URI'] = 'http://google.com/'

client_id = config('CLIENT_ID')
client_secret = config('CLIENT_SECRET')
redirect_uri = config('REDIRECT_URI')

In [ ]:
# authorization
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

In [ ]:
username = 'sidsaxena'

In [ ]:
scope = 'ugc-image-upload user-read-playback-state user-modify-playback-state user-read-currently-playing streaming app-remote-control user-read-email user-read-private playlist-read-collaborative playlist-modify-public playlist-read-private playlist-modify-private user-library-modify user-library-read user-top-read user-read-playback-position user-read-recently-played user-follow-read user-follow-modify'

In [ ]:
def getToken():
    try:

        token = util.prompt_for_user_token(username=username, scope=scope, client_id=client_id, client_secret= client_secret, redirect_uri=redirect_uri, cache_path='.cache-sidsaxena')
        sp = spotipy.Spotify(auth=token)

    except:
        print('Token not accessible for user: ', username)
    return sp

In [ ]:
sp = getToken()

## Define Functions to parse info

In [ ]:
def getUserPlaylists(user):

    name_list = []
    id_list = []
    creator_list = []
    offset=0
    playlists = []

    while True:
        results = sp.user_playlists(user, offset=offset)
        playlists += results['items']

        if results['next'] is not None:
            offset += 50
        else:
            break

    for playlist, _ in enumerate(playlists):
        name_list.append(playlists[playlist]['name'])
        id_list.append(playlists[playlist]['id'])
        creator_list.append(playlists[playlist]['owner']['id'])
    
    creator_id_tuple = tuple(zip(creator_list, id_list))

    playlist_dict = dict(name = name_list, id = id_list, creator = creator_list)
        
    playlist_df = pd.DataFrame(playlist_dict)
    multiple_playlist_dict = list(creator_id_tuple)
    playlist_df.to_csv('{}-playlists.csv'.format(user))
    return playlist_df, multiple_playlist_dict

In [ ]:
def getTrackIds(user, playlist_id):
    """get track ids from a playlist and return id list."""
    
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

# ids = getTrackIds(my_user, vfar_id)

In [ ]:
def getTrackIdsFromAlbum(album_id):

    tracklist = []
    track_id_list = []

    results = sp.album_tracks(album_id)
    tracks = results['items']
    album_results = sp.album(album_id)
    album_name = album_results['name'] 
    album_tracks = album_results['tracks']
    for track, _ in enumerate(tracks):
        tracklist.append(tracks[track]['name'])
        track_id_list.append(tracks[track]['id']) 

    return track_id_list

In [ ]:
def analysePlaylist(creator, playlist_id):
    
    tracks = []
    offset = 0
    # Create empty dataframe
    playlist_features_list = ['artist','album','track_name', 'track_id', 'popularity', 'genres', 'danceability','energy','key','loudness','mode', 'speechiness','instrumentalness','liveness','valence','tempo', 'duration_ms','time_signature']
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
        
    while True:
        results = sp.user_playlist_tracks(creator, playlist_id, offset=offset)
        tracks += results['items']

        if results['next'] is not None:
            offset += 100
        else:
            break

    for track in tracks:       
        
        artist = track['track']['album']['artists'][0]
        artist_id = artist['id']
        genre = sp.artist(artist_id)
        genres = genre['genres']

        playlist_features = {}        
        playlist_features['artist'] = artist['name']
        playlist_features['album'] = track['track']['album']['name']
        playlist_features['track_name'] = track['track']['name']
        playlist_features['track_id'] = track['track']['id']
        playlist_features['popularity'] = track['track']['popularity']
        playlist_features['genres'] = [genres]
    
        # Get audio features
        time.sleep(0.5)
        audio_features = sp.audio_features(playlist_features['track_id'])[0]
        for feature in playlist_features_list[6:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
    
    playlist_df.to_csv('{}-{}.csv'.format(creator, playlist_id))
    return playlist_df

In [ ]:
def analysePlaylistsList(playlist_tuple_list):
    """function to analyse multiple playlists"""

    for id, _ in enumerate(playlist_tuple_list):
        playlist_df = analysePlaylist(playlist_tuple_list[id][0], playlist_tuple_list[id][1])
        # playlist_df['playlist'] = PLAYLIST NAME

        if id == 0:
            playlist_tuple_df = playlist_df
        else:
            playlist_tuple_df = pd.concat([playlist_tuple_df, playlist_df], ignore_index=True)

    playlist_tuple_df.to_csv('multiple playlists.csv')
    return playlist_tuple_df

In [ ]:
def getDiscography(name):
    """get discography of an artist by searching its name and return a dataframe and genres in a seperate variable"""

    release_date_list = []
    track_id_list = []   
    tracklist = []
    album_list = []    
    track_album_dict = {}

    album_names = []
    album_id = []
    album_release_date = []

    single_names = []
    single_id = []
    single_release_date = []

    results = sp.search(q=name, type='artist')
    artist = results['artists']['items'][0]

    artist_name = artist['name']
    artist_id = artist['id']
    artist_genres = artist['genres']

    artist_albums = sp.artist_albums(artist_id, album_type='album')
    albums = artist_albums['items']
    
    while artist_albums['next']:
        artist_albums = sp.next(artist_albums)
        albums.extend(artist_albums['items'])

    for album in albums:
        album_names.append(album['name'])
        album_id.append(album['id'])
        album_release_date.append(album['release_date'])

    
    artist_singles = sp.artist_albums(artist_id, album_type='single')
    singles = artist_singles['items']
    
    while artist_singles['next']:
        artist_singles = sp.next(artist_singles)
        singles.extend(artist_singles['items'])

    for single in singles:
        single_names.append(single['name'])
        single_id.append(single['id'])
        single_release_date.append(single['release_date'])


    # MUCH BETTER CODE but need to figure out a way to append release date.
    # for album in album_id:
    #     tracks = sp.album_tracks(album)
    #     tracks = tracks['items']
    #     album_name = sp.album(album)['name']
    #     for track in tracks:
    #         track_name = track['name']
    #         track_id_list.append(track['id'])
    #         tracklist.append(track_name)
    #         album_list.append(album_name)
    #         release_date_list.append(album_release_date[])


    for album, _ in enumerate(album_id):
        tracks = sp.album_tracks(album_id[album])
        tracks = tracks['items']
        album_name = sp.album(album_id[album])['name'] 
        for track, _ in enumerate(tracks):
            track_name = tracks[track]['name']
            track_id_list.append(tracks[track]['id']) 
            tracklist.append(track_name)
            album_list.append(album_name)
            release_date_list.append(album_release_date[album])
    
    for single, _ in enumerate(single_id):
        tracks = sp.album_tracks(single_id[single])
        tracks = tracks['items']
        album_name = sp.album(single_id[single])['name'] 
        for track, _ in enumerate(tracks):
            single_name = tracks[track]['name']
            track_id_list.append(tracks[track]['id']) 
            tracklist.append(single_name)
            album_list.append(album_name)
            release_date_list.append(single_release_date[single])

    


    track_album_dict = {'track': tracklist, 'album': album_list, 'release_date': release_date_list, 'id': track_id_list}  
    df = pd.DataFrame(track_album_dict)
    df.to_csv('{}-discog.csv'.format(artist['name']))
    return df

In [ ]:
def getTrackFeatures(id):
    """get features of a track by its ID."""
    
    meta = sp.track(id)
    features = sp.audio_features(id)

    #meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    artist_id = meta['album']['artists'][0]['id']
    genre = sp.artist(artist_id)['genres']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    #features
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    key = features[0]['key']
    loudness = features[0]['loudness']
    mode = features[0]['mode']
    speechiness = features[0]['speechiness']
    acousticness = features[0]['acousticness']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    valence = features[0]['valence']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

    track = [name, album, artist, release_date, genre, length, popularity,danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature]

    return track

In [ ]:
def getTracklistFeatures(tracklist):
    """get features of multiple tracks from a list of IDs and return a dataframe"""

  # loop over track ids 
    tracks = []
    for id in range(len(tracklist)):
        track = getTrackFeatures(tracklist[id])
        tracks.append(track)
        
        # see status
        print('Requesting track {}/{}'.format(id, len(tracklist)))
        # clear output
        clear_output(wait=True)

  # create dataset
    df = pd.DataFrame(tracks, columns = ['track', 'album', 'artist', 'release_date', 'genres', 'length', 'popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'])
    df.to_csv("tracklist-features.csv", sep = ',')
    return df

In [ ]:
def getUriFromDf(df):
    saved_uris = []
    failed_tracks = []
    artist_names = df['artist'].values
    track_names = df['track'].values

    for i in range(len(artist_names)):
        artist = artist_names[i]
        track = track_names[i]
        print(f'{i} - {artist}: {track}')
        q = 'artist:{} track: {}'.format(artist, track)
        results = sp.search(q=q, limit=1, type='track')
        if len(results['tracks']['items']) > 0:
            uri = results['tracks']['items'][0]['uri']
            saved_uris.append(uri)
        else: 
            failed_tracks.append(f'{i} - {artist}: {track}')
            i += 1        
    return saved_uris

## Get playlists and their features

### Get all playlists

In [ ]:
my_playlists_df, my_playlists_list = getUserPlaylists(username)

In [ ]:
my_playlists_df

In [ ]:
# first 10 playlists
my_playlists_list[:10]

## Get uris of scrobbles spreadsheet 

In [ ]:
# scrobbles_uri = getUriFromDf(scrobbles_unique)

In [ ]:
# l = list(csv.reader(open('../spotify/Spreadsheets/scrobbles_unique_uris.csv', 'r')))
# mylist = map(list, zip(*l[0:])) # transpose list
# scrobble_uris = list(chain.from_iterable(mylist))

In [ ]:
# np.savetxt("scrobbles_uri_unique.csv", saved_uris, delimiter=",", fmt='%s')

In [ ]:
# scrobble_features = getTracklistFeatures(scrobble_uris)
# scrobbles_features.to_csv('scrobbles_features.csv', index=False)

In [ ]:
# refresh token
sp = getToken()

In [ ]:
scrobbles = pd.read_csv('Spreadsheets/SidSaxena_scrobbles_2020-08-25.csv')
scrobbles = scrobbles.drop(['album', 'artist_mbid', 'album_mbid', 'track_mbid', 'timestamp'], axis=1)

In [ ]:
scrobbles['artist'] = scrobbles['artist'].apply(lambda x: string.capwords(x))
scrobbles['track'] = scrobbles['track'].apply(lambda x: string.capwords(x))

In [ ]:
scrobbles

In [ ]:
scrobbles_unique = pd.read_csv('Spreadsheets/SidSaxena_unique_scrobbles_2020-08-25.csv')
scrobbles_unique

In [ ]:
scrobbles_features = pd.read_csv('Spreadsheets/Scrobbles Features.csv')
scrobbles_features = scrobbles_features.drop_duplicates()
scrobbles_features.rename(columns={'name':'track'}, inplace=True)
scrobbles_features

In [ ]:
# # add seatbelts features to scrobbles features
# seatbelts = scrobbles_unique[scrobbles_unique['artist'] == 'The Seatbelts']

# seatbelts_uri = []
# artist = seatbelts['artist'].to_list()
# tracks = seatbelts['track'].to_list()

# l = [i.split('The ', 1)[1] for i in artist]

# for i in range(len(l)):
#     artist = l[i]
#     track = tracks[i]
#     q = 'artist:{} track: {}'.format(artist, track)
#     print(f'{i} - {artist}: {track}')
#     results = sp.search(q=q, limit=1, type='track')
#     if len(results['tracks']['items']) > 0:
#         uri = results['tracks']['items'][0]['uri']
#         seatbelts_uri.append(uri)
#     else: 
#         i += 1

# seatbelts_features = getTracklistFeatures(seatbelts_uri)
# scrobbles_features.append(seatbelts_features)

In [ ]:
scrobbles_features['artist'] = scrobbles_features['artist'].apply(lambda x: string.capwords(x)) 
scrobbles_features['track'] = scrobbles_features['track'].apply(lambda x: string.capwords(x))

## Merge scrobbles and features

In [ ]:
merged = pd.merge(scrobbles, scrobbles_features)
merged = merged.sort_values('datetime', ascending=False)

In [ ]:
merged

In [ ]:
# merged.to_csv('Scrobbles with Features (66K).csv', index=False)

In [ ]:
mergeunion = pd.merge(scrobbles, scrobbles_features, how='outer')

In [ ]:
# mergeunion.to_csv('Union of Scrobbles and Features.csv', index=False)

In [ ]:
mergeunion

### Deal with failed merge

In [ ]:
failed = pd.concat([mergeunion, merged, merged]).drop_duplicates(keep=False)

In [ ]:
# failed.to_csv("Failed Results (Union minus Merged).csv", index=False)

In [ ]:
failed

In [ ]:
failed_not_features = failed[failed['length'].isnull()]

failed_not_features = failed_not_features[['artist', 'track', 'datetime']]

In [ ]:
# failed_not_features.to_csv('Failed Not Features.csv', index=False)

In [ ]:
failed_features = failed[failed['length'].notnull()]

failed_features = failed_features.drop('datetime', axis=1)

In [ ]:
# failed_features.to_csv('Failed Features.csv', index=False)

In [ ]:
failed_features

In [ ]:
same_track = pd.merge(failed_features, failed_not_features, on='track')

In [ ]:
same_track['artist_x'] = same_track['artist_y']

In [ ]:
same_track.drop(['artist_y'], axis=1, inplace=True)

In [ ]:
same_track.rename(columns={'artist_x': 'artist'}, inplace=True)

In [ ]:
same_track

In [ ]:
failed_merge = pd.merge(failed_not_features, same_track)
# failed_merge.drop('artist_x', axis=1, inplace=True)

In [ ]:
failed_merge

In [ ]:
# failed_merge.to_csv('Failed Merge.csv', index=False)

In [ ]:
new_merged = pd.concat([merged, failed_merge])
new_merged.drop_duplicates(inplace=True)

In [ ]:
new_merged

In [ ]:
# new_merged.to_csv('Scrobbles with Features (69K).csv', index=False)

In [ ]:
# scrobbles_features['artist'] = scrobbles_features['length'].map(test_dict).fillna(scrobbles_features['artist'])

# scrobbles_features['artist'] = scrobbles_features['artist'].apply(lambda x: artists_dict[x])

# df['Group'] = df['Group'].map(df1.set_index('Group')['Hotel'])

# hotel_dict = df2.set_index('Group').to_dict()
# df1['Group'] = df1['Group'].apply(lambda x: hotel_dict[x])

## Merge Exact_Merges from Record Linkage (Linking.ipynb)

In [ ]:
exact_merge = pd.read_csv('Spreadsheets/Exact Merge.csv')

In [ ]:
new_merged = pd.concat([new_merged, exact_merge]).drop_duplicates()

In [ ]:
new_merged = new_merged.drop_duplicates(subset='datetime', keep="first")

In [ ]:
new_merged

In [ ]:
new_merged = new_merged.sort_values(by='datetime', ascending=False)

In [ ]:
# new_merged.to_csv('Scrobbles with Features (69126).csv', index=False)

# Get certain playlists and their attributes

In [75]:
my_playlists_df

,name,id,creator
0,Ear training: Intervals - Perfect Prime,4TCFTC76ZJ5MoirLeicr3g,0zxrr00u8c7ronq8dmkkgzsfr
1,Ear training: Intervals - Major Second,03dSOaehYRlETqEeHUvaur,0zxrr00u8c7ronq8dmkkgzsfr
2,Discover Weekly Archive,2HMHUoxUgMcyBIhc2AigWN,sidsaxena
3,Byronheart Blended Daily,7JYmWD4GmCA72LM1rMENjZ,sidsaxena
4,Byronheart + Gadha,37i9dQZF1EJwk8bMFJ86Hp,spotify
5,Dhruv + Gadha,37i9dQZF1EJDAofyoIEVRO,spotify
6,Samarth + Gadha,37i9dQZF1EJzLXmaJSWoa8,spotify
7,Twoheadedboy + Gadha,37i9dQZF1EJA9FtsyReGLU,spotify
8,woosteresque + Gadha,37i9dQZF1EJMC6i7vRuhmV,spotify
9,Nanda + Gadha,37i9dQZF1EJu04AyFmTvOk,spotify


### k not so pop (manually)

In [77]:
my_playlists_df[my_playlists_df['name'].str.contains('not')]

,name,id,creator
28,k not so pop,6IEGMTxfOIFo7osyuHCdsm,sidsaxena
30,Voices from Another Room,2CTJSu2bawrJlyeWJtOKYC,sidsaxena
55,Hindi music that's not great but also not shit,0QIWliqImR4r7h8HHkXhj6,ahf4gi6ob4f0i80fkool0shl4


In [78]:
kindie = analysePlaylist(my_playlists_df.iloc[28, 2], my_playlists_df.iloc[28, 1])

kindie

# kindie.to_csv('Spreadsheets\Playlists\k not so pop.csv', index=False)

,artist,album,track_name,track_id,popularity,genres,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Fuyumi Abe,沈黙の恋人,希望のうた,24dq1uvUoVVQ6WMwFTqUPl,37,[j-acoustic],0.384,0.0751,2,-16.534,1,0.0436,0.000001,0.1320,0.254,162.503,245747,3
1,The Black Skirts,Hollywood,Hollywood,4kqN6K6J1nM18zivvFLx8v,17,"[k-indie, k-rock, korean dream pop, korean ind...",0.552,0.7820,1,-8.082,0,0.0302,0.900000,0.1000,0.509,147.956,331000,4
2,HYUKOH,24 : How to find true love and happiness,LOVE YA!,7rDdN55Ma9gZlbexHiDSud,23,"[k-indie, korean indie folk, korean r&b]",0.375,0.5660,11,-6.527,1,0.0448,0.000016,0.1460,0.330,143.770,251965,4
3,The Chairs,LOVELY SUNDAY,Rollin' On,63oSPrP54in4X41vq6cNlL,43,[chinese indie],0.687,0.5560,5,-9.366,1,0.0286,0.018900,0.1820,0.517,103.907,234600,4
4,ADOY,LOVE,Wonder,0v89jw9S9NPzOPCoZnLZxE,2,"[k-indie, korean city pop, korean dream pop, k...",0.661,0.7370,6,-6.317,1,0.0287,0.544000,0.1150,0.532,99.989,235747,4
5,ADOY,CATNIP,Grace,6hpWk3kV9phxtzxbzzXmIF,47,"[k-indie, korean city pop, korean dream pop, k...",0.622,0.7010,1,-6.273,0,0.0312,0.000028,0.0965,0.638,100.007,245280,4
6,The Chairs,The Chairs on Audiotree Live,Rollin' On - Audiotree Live Version,2oPgiaiXfijOeDAeA7eL4C,13,[chinese indie],0.686,0.4660,5,-10.798,1,0.0257,0.000182,0.1710,0.626,103.982,232233,4
7,Various Artists,CINEMA SESSIONS 2019,Rollin' On,12HcAJuzuH2Rr9xBNJ98t2,5,[],0.705,0.5760,5,-8.043,1,0.0289,0.000009,0.3500,0.558,104.056,244629,4
8,Sunday Moon,City Lights,City Lights,36xcAYhiCUnzBoydVmxkO3,45,[k-indie],0.602,0.3030,6,-7.978,1,0.0451,0.091000,0.1450,0.327,135.990,241699,4
9,SURL,Aren't You?,The Lights Behind You,6O1OrHXMWpJbhmWSJnncw1,1,"[k-indie, korean indie rock]",0.492,0.5500,9,-3.421,1,0.0320,0.001840,0.1500,0.382,80.987,312640,4


### voices from another room (semi-automatic)

In [106]:
vfar = my_playlists_df[my_playlists_df['name'].str.contains('Voices')].iloc[0]
vfar

name       Voices from Another Room
id           2CTJSu2bawrJlyeWJtOKYC
creator                   sidsaxena
Name: 30, dtype: object

In [109]:
vfar_features = analysePlaylist(vfar['creator'], vfar['id']) # (creator, id)

In [110]:
# vfar_features.to_csv('Spreadsheets/Playlists/Voices From Another Room.csv', index=False)

In [111]:
vfar_features

,artist,album,track_name,track_id,popularity,genres,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Barrie,Happy To Be Here,Geology,4qCfbDTCrFnfG5fYngu2wE,48,"[brooklyn indie, indie pop]",0.690,0.721,4,-8.437,1,0.0323,0.649000,0.1250,0.3570,104.027,268120,4
1,"Astronauts, etc.",Mind Out Wandering,Up for Grabs,2gyXXC4IMj8K4HfUNUvqKO,24,"[bay area indie, shiver pop]",0.539,0.525,5,-10.910,1,0.0285,0.234000,0.1050,0.5350,175.972,176627,4
2,CASTLEBEAT,CASTLEBEAT,Change Your Mind,4omurqpm7aWH9VVz2Ii4yO,53,"[bedroom pop, dreamo, indie garage rock, indie...",0.384,0.578,11,-9.624,1,0.0312,0.886000,0.1250,0.0984,150.013,297273,4
3,You'll Never Get to Heaven,Images,Beyond the Clouds,6vMaCzTnismT6N45MDCMfa,39,"[ambient pop, london on indie, nu gaze]",0.633,0.575,7,-9.667,0,0.0298,0.924000,0.1050,0.2390,117.520,248681,4
4,Day Wave,Drag,Drag,0oXwUeFAE6IR2lYhaTpxRT,59,"[bay area indie, chillwave, dreamo, indie pop,...",0.537,0.713,7,-6.250,1,0.0272,0.650000,0.1000,0.3670,93.198,189651,4
5,Voka Gentle,Branscombe,Branscombe,4mrOXzvbS0U0KsX7O6OqIo,42,[],0.556,0.389,7,-15.394,1,0.0290,0.887000,0.1110,0.0865,111.012,439478,4
6,Acid Ghost,All Alone,All Alone,72xnRXxHiAhTfKSzlPdEmo,54,[indie garage rock],0.509,0.588,0,-9.542,1,0.0298,0.845000,0.1150,0.4870,171.845,168648,4
7,The F16s,Amber,Amber,62bHWlIlUW6fbwBO29h64H,36,"[chennai indie, indian indie, indian rock]",0.299,0.468,2,-10.777,1,0.0259,0.770000,0.2240,0.5930,95.069,206704,4
8,The Shutes,Echo of Love,Bright Blue Berlin Sky,6P30NMoLLzWTuue4faRjsd,42,[isle of wight indie],0.534,0.626,6,-8.549,0,0.0288,0.034500,0.1090,0.3770,120.920,344633,4
9,Electric Youth,Memory Emotion,Breathless,6bgUlpf4HVpuTdo5LuBTov,34,"[filter house, popwave, synthwave]",0.618,0.623,2,-7.899,1,0.0267,0.841000,0.1020,0.3330,97.984,271683,4


### blends

In [124]:
blends = my_playlists_df[my_playlists_df['name'].str.contains('\+')]
blends

,name,id,creator
4,Byronheart + Gadha,37i9dQZF1EJwk8bMFJ86Hp,spotify
5,Dhruv + Gadha,37i9dQZF1EJDAofyoIEVRO,spotify
6,Samarth + Gadha,37i9dQZF1EJzLXmaJSWoa8,spotify
7,Twoheadedboy + Gadha,37i9dQZF1EJA9FtsyReGLU,spotify
8,woosteresque + Gadha,37i9dQZF1EJMC6i7vRuhmV,spotify
9,Nanda + Gadha,37i9dQZF1EJu04AyFmTvOk,spotify
10,Shashwat + Gadha,37i9dQZF1EJx5ncR2PUGOj,spotify
11,Udit + Gadha,37i9dQZF1EJJnrCJLcfFKP,spotify


In [131]:
byron = blends.iloc[0]
byron_blend = analysePlaylist(byron['creator'], byron['id'])
byron_blend

,artist,album,track_name,track_id,popularity,genres,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Day Wave,Drag,Drag,0oXwUeFAE6IR2lYhaTpxRT,59,"[bay area indie, chillwave, dreamo, indie pop,...",0.537,0.713,7,-6.250,1,0.0272,0.650000,0.1000,0.3670,93.198,189651,4
1,Sleigh Bells,Treats,Rill Rill,3GbjOImkNo5nFT2GQxVr11,54,"[alternative dance, art pop, brooklyn indie, e...",0.468,0.812,1,-3.440,0,0.0491,0.000001,0.0688,0.7890,72.987,229893,4
2,Alvvays,Antisocialites,Not My Baby,27bBjan9bD8rUESGYqK9a7,54,"[dreamo, indie pop, indie rock, shoegaze, toro...",0.342,0.644,0,-4.363,1,0.0346,0.299000,0.0988,0.1290,150.848,256413,4
3,St. Vincent,MASSEDUCTION,New York,5IXTT9RvcVupmzLTFqIInj,56,"[art pop, electropop, etherpop, freak folk, in...",0.373,0.449,10,-11.646,1,0.1440,0.000013,0.1750,0.3270,137.866,154960,4
4,John Mayer,Heavier Things,Bigger Than My Body,0Jh9EBGN205Yc7JN2zZ2f1,57,"[neo mellow, pop, pop rock, singer-songwriter]",0.608,0.937,4,-3.071,1,0.0330,0.040300,0.3090,0.5150,117.304,265960,4
5,Steven Wilson,THE FUTURE BITES,12 THINGS I FORGOT,3U5qkg3OoX0KQxEaGqLYQT,32,"[art rock, progressive metal, progressive rock...",0.603,0.677,9,-9.395,1,0.0275,0.000115,0.0907,0.2170,119.057,282778,4
6,Halsey,BADLANDS (Live From Webster Hall),Gasoline,4k0yY041y3wLd41uDjqROv,37,"[dance pop, electropop, etherpop, indie poptim...",0.731,0.580,10,-7.328,0,0.0399,0.000000,0.1290,0.3190,120.001,199593,4
7,The Paper Kites,twelvefour,Revelator Eyes,4A8wLC09mcPYI6ZwM6Ia44,42,"[australian indie folk, indie folk, stomp and ...",0.520,0.717,8,-9.016,1,0.0299,0.054000,0.1730,0.5680,153.025,262267,4
8,They Might Be Giants,Flood,Istanbul (Not Constantinople),63vL5oxWrlvaJ0ayNaQnbX,65,"[alternative rock, comic, geek rock, permanent...",0.688,0.748,5,-9.390,0,0.0377,0.004530,0.1360,0.8920,114.144,153813,4
9,Yuko Ando,Shogeki,Shogeki,5QwAdWCnauw0GpDzM3glrO,66,"[anime, japanese singer-songwriter]",0.399,0.723,11,-4.507,1,0.0681,0.000000,0.3520,0.2200,137.032,181595,4


In [135]:
byron_blend.to_csv("Spreadsheets/Playlists/Byronheart + Gadha.csv", index=False)

In [140]:
test = getTracklistFeatures(['0IcSLT53eE07Jmok64Ppo3'])
test.to_csv('../Other/test.csv', index=False)

Requesting track 0/1


In [137]:
sp = getToken()